In [ ]:
import os,sys
WORKSPACE_DIR = '../..' 
PROJECT_DIR = f'{WORKSPACE_DIR}/barchartapi' 
WORKING_DIR = f'{PROJECT_DIR}/barchartapi' 
if WORKING_DIR not in sys.path:
    sys.path.append(WORKING_DIR)
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)
if WORKSPACE_DIR not in sys.path:
    sys.path.append(WORKSPACE_DIR)    
import barchart_api as bcapi
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib import gridspec
%matplotlib inline
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.tools as tls
init_notebook_mode(connected=True)

import datetime
import pandas as pd
import plot_utilities as pu
import datetime
import numpy as np



In [ ]:
import importlib
importlib.reload(pu)

In [ ]:
# set this to 'free' or 'paid'
endpoint = 'free' # free or paid

# set the bar_type and the interval
bar_type='daily' # minutes, daily, monthly
interval=1 # 1,5,15,30,60

# create an instance 
api_key = open(f'./temp_folder/{endpoint}_api_key.txt','r').read()
endpoint_type=f'{endpoint}_url'
bch = bcapi.BcHist(api_key, bar_type=bar_type, interval=interval,endpoint_type = endpoint_type)


In [ ]:
df_cb = bch.get_history('CLU19',20190610,20190628)


In [ ]:
df_cb[1]

In [ ]:
yy = str(datetime.datetime.now().year)[2:]
mm = datetime.datetime.now().month
if mm > 6:
    yy = str(int(yy) + 1)
short_name = f'CLZ{yy}'
days_to_fetch = 120

dt_end = datetime.datetime.now()
dt_beg = dt_end - datetime.timedelta(days_to_fetch)
beg_yyyymmdd = '%04d%02d%02d' %(dt_beg.year,dt_beg.month,dt_beg.day)#20181201
end_yyyymmdd = '%04d%02d%02d' %(dt_end.year,dt_end.month,dt_end.day)#20190219

tup = bch.get_history(short_name, beg_yyyymmdd, end_yyyymmdd)
print(f'get_history: {short_name} WRITING DATA {datetime.datetime.now()}')
df_hist = tup[1]
def __full_date(d):
    year = int(str(d)[0:4])
    month = int(str(d)[5:7])
    day = int(str(d)[8:10])
    hour = int(str(d)[11:13])
    minute = int(str(d)[14:16])
    dt = datetime.datetime(year,month,day,hour,minute)
    return(dt)
    
df_hist['date'] = df_hist.timestamp.apply(__full_date)
print(f'get_history: {short_name} END {datetime.datetime.now()}')
df_hist.tail()

___
### Create Candlestick charts for each day's 30 minute intra-day data
___

In [ ]:
if endpoint != 'free':
    for day in df_hist.tradingDay.unique():
        try:
            fig,axs = pu.candles(df_hist[df_hist.tradingDay==day])
            plt.rcParams.update({'figure.max_open_warning': 0})
        except Exception as e:
            print(f'ERROR FOR DAY {day}')
            print(str(e))


### Show a barchart of the  daily history of percent changes between CLX19 and XLE

In [ ]:
status,df_clx19 = bch.get_history('CLX19',20190801,20191001)
status,df_xle = bch.get_history('XLE',20190801,20191001)
df_clx19['pct_cl'] = df_clx19.close.pct_change()
df_xle['pct_xle'] = df_xle.close.pct_change()

In [ ]:
df_both = df_clx19[['tradingDay','pct_cl']].merge(df_xle[['tradingDay','pct_xle']],on='tradingDay',how='inner')

In [ ]:
f = pu.plotly_pandas(df_both,'tradingDay',bar_plot=True,alt_yaxis=False)
iplot(f)

In [ ]:
f2 = pu.plot_pandas(df_both,'tradingDay',bar_plot=True,alt_yaxis=False)

In [ ]:
dir(f2.get_yaxis())

In [ ]:
df_both[[c for c in df_both.columns.values if c != 'tradingDay']]

### End